In [1]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import numpy as np
import time 
import json

In [2]:
#Scrape the Season Schedule
year_url = "https://www.footballdb.com/games/index.html?lg=NFL&yr=2016"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
browser.visit(year_url)
year_html = browser.html
year_soup = BeautifulSoup(year_html, 'html.parser')
print ("--- Chrome Driver Complete ---")

--- Chrome Driver Complete ---


In [3]:
#create schedule of full season with date and URL for Box Score
full_schedule = []
tables = year_soup.findAll("table", class_="statistics")
for x in np.arange(len(tables)):
    table = tables[x].findAll("tbody")
    for t in table:
        rows = t.findAll("tr")
        for row in rows:
            d = row.findAll("td")
            date = d[0].text[:10]
            for a in row.findAll("a", href = True):
                gameURL = a["href"]
            full_schedule.append({"date":date, "gameURL": gameURL})
#             print (f'{date} // {gameURL} URL acquired')
print (full_schedule)

[{'date': '09/08/2016', 'gameURL': '/games/boxscore.html?gid=2016090801'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091101'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091102'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091103'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091104'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091105'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091106'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091107'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091108'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091109'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091110'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091111'}, {'date': '09/11/2016', 'gameURL': '/games/boxscore.html?gid=2016091112'}, {'date': '09/11/2016', 'gameURL': '/g

In [9]:
#Iterate through URLS
stats_db = []
for game in full_schedule:
#     print(game)
    gameday = game['date']
    url_ending = game['gameURL']
    game_url = "https://www.footballdb.com" + url_ending
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless = True)
    browser.visit(game_url)
    game_html = browser.html
    game_soup = BeautifulSoup(game_html, 'html.parser')
    left_table = game_soup.findAll("div", class_="section_left")[0].findAll("tr")
    right_table = game_soup.findAll("div", class_="section_right")[0].findAll("tr")
    score_table = game_soup.findAll("table", class_="statistics")[0].findAll("tr")
    print ("--- Game URL Scraping ---")
    
    # Create Dictionaries for game by team
    
    away_data = []
    home_data = []
    for row in left_table:
        tableData = row.findAll("td")
        away_data.append(tableData[1].text)
        home_data.append(tableData[2].text)
    for row in right_table[1:]:
        tableData = row.findAll("td")
        away_data.append(tableData[1].text)
        home_data.append(tableData[2].text)
    awayScore = int(score_table[1].findAll("td")[5].text)
    homeScore = int(score_table[2].findAll("td")[5].text)
    totalScore = awayScore + homeScore
    awayMargin = awayScore - homeScore
    homeMargin = homeScore - awayScore
    if homeMargin < 0:
        awayResult = "Win"
        homeResult = "Loss"
    elif homeMargin == 0:
        awayResult = "Tie"
        homeResult = "Tie"
    elif homeMargin > 0:
        awayResult = "Loss"
        homeResult = "Win"
        
# Create Stats DataBase

    
    away_stats = {"date": gameday,
        "team": away_data[0],
        "points": awayScore,
        "opp": home_data[0],
        "points_allowed": homeScore,
        "total_points": totalScore,
        "margin": awayMargin,
        "result": awayResult,
        "first_downs": int(away_data[1]),
        "total_yards": int(away_data[5]),
        "rush_yards": int(away_data[6]),
        "pass_yards": int(away_data[9]),
        "penalty_yards": int(away_data[19].split("-")[1]),
        "sacks": int(home_data[11].split("-")[0]),
        "takeaways": int(home_data[10].split("-")[2])+int(home_data[20].split("-")[1]),
        "3d%": float(away_data[22].split("-")[0])/float(away_data[22].split("-")[1]),
        "plays": int(away_data[24]),
        "TOP": float(away_data[26].split(":")[0])+(float(away_data[26].split(":")[1])/60),
        "first_downs_allowed": int(home_data[1]),
        "total_yards_allowed": int(home_data[5]),
        "rush_yards_allowed": int(home_data[6]),
        "pass_yards_allowed": int(home_data[9]),
        "sacked": int(away_data[11].split("-")[0]),
        "turnovers": int(away_data[10].split("-")[2])+int(away_data[20].split("-")[1]),
        "3d%_allowed": float(home_data[22].split("-")[0])/float(home_data[22].split("-")[1])
    }
#     print(away_stats)
    home_stats = {"date": gameday,
        "team": home_data[0],
        "points": homeScore,
        "opp": away_data[0],
        "points_allowed": awayScore,
        "total_points": totalScore,
        "margin": homeMargin,
        "result": homeResult,
        "first_downs": int(home_data[1]),
        "total_yards": int(home_data[5]),
        "rush_yards": int(home_data[6]),
        "pass_yards": int(home_data[9]),
        "penalty_yards": int(home_data[19].split("-")[1]),
        "sacks": int(away_data[11].split("-")[0]),
        "takeaways": int(away_data[10].split("-")[2])+int(away_data[20].split("-")[1]),
        "3d%": float(home_data[22].split("-")[0])/float(home_data[22].split("-")[1]),
        "plays": int(home_data[24]),
        "TOP": float(home_data[26].split(":")[0])+(float(home_data[26].split(":")[1])/60),
        "first_downs_allowed": int(away_data[1]),
        "total_yards_allowed": int(away_data[5]),
        "rush_yards_allowed": int(away_data[6]),
        "pass_yards_allowed": int(away_data[9]),
        "sacked": int(home_data[11].split("-")[0]),
        "turnovers": int(home_data[10].split("-")[2])+int(home_data[20].split("-")[1]),
        "3d%_allowed": float(away_data[22].split("-")[0])/float(away_data[22].split("-")[1])
    }
#     print(home_stats)
    stats_db.append(away_stats)
    stats_db.append(home_stats)
    print (f'{away_stats["team"]} vs. {home_stats["team"]} on {gameday} complete!')

print (stats_db)

--- Game URL Scraping ---
CarolinaCar vs. DenverDen on 09/08/2016 complete!
--- Game URL Scraping ---
Tampa BayTB vs. AtlantaAtl on 09/11/2016 complete!
--- Game URL Scraping ---
MinnesotaMin vs. TennesseeTen on 09/11/2016 complete!


TimeoutException: Message: timeout
  (Session info: headless chrome=67.0.3396.87)
  (Driver info: chromedriver=2.36.540470 (e522d04694c7ebea4ba8821272dbef4f9b818c91),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
nfl_2016 = pd.DataFrame(stats_db)
nfl_2016.head()

In [ ]:
nfl_2016.to_csv("nfl_2016.csv")